# Quantum Hikers

Gathering from both sides of the rough Mont Blanc, a team of italian and french junior quantum scientists decided to merge their super-quantum powers to dominates the mountains... Here are our approaches to determine the best way through the thoughest journeys.

Team members : Fabrice Monasterio, Emanuele Albertini, Andrea Basilio Rava, Jeanne Bourgeois

## Software part

The goal of this part is to expand our skills on QAOA and AQO algorithms to tackle a Traveling Salesman Problem. The key point here is to efficiently encode our problem in Qibo variables and Hamiltonians...

### Getting in shape : a first tutorial challenge

We start our challenge by considering a weighted complete graph of N=3 nodes. We seak for an ordering of these 3 nodes that would describe the shortest path visiting once each of these nodes. 

#### Hamiltonian encoding of the problem

To do so, we introduce the binary variables $x_{i,t}$, where $i\in\{0,1,2\}$ scans the nodes of the graphs and $t\in\{0,1,2\}$ the time steps of the path, such that $x_{i,t}=1$ if the path visits the node $i$ at time $t$, and $0$ otherwise.

The length of the path is encoded in an Hamiltonian form as:
$$H_{\text{dist}}=\sum_{i,j=0}^{N-1} w_{i,j} \sum_{t=0}^{N-2} x_{i,t}x_{j,t+1}.$$

We impose to our path to visit every node exactly once, which is statisfied by minimising the Hamiltonian term:
$$H_{\text{nodes}}=\sum_{i=0}^{N-1}(1-\sum_{t=0}^{N-1}x_{i,t})^2,$$
and to be at each step at exactly one node, which is satisfied by minimising the Hamiltonian term:
$$H_{\text{steps}}=\sum_{t=0}^{N-1}(1-\sum_{i=0}^{N-1}x_{i,t})^2.$$

We will thus carry on our protocols considering the following cost Hamiltonian :
$$H_{\text{cost}} = H_{\text{dist}} + A * (H_{\text{nodes}} + H_{\text{steps}}),$$
where $A$ is a penalty coefficient that we should set big enough so that the constraints are strictly respected, but not too bigh either as the optimal solution would be harder to find. Initially, we set $A=100$, which is 2 orders of magnitude bigger than the weight coefficients. Finally, we went for smaller values of $A$.

To fit with the encoding of our quantum devices, we perform the following change of variables:
$$x_{i,t} = \frac{1-z_{i,t}}{2}$$
so that we will work with $z_{i,t}$ in $\{-1, 1\}.


#### Carrying out the optimisation

To explore the space of feasible solution, we use the following mixing matrix:
$$H_{\text{mixing}} = \sum_{i,t=0}^{N} X_{i, t}$$
and we start the annealing protocol in its ground state $\text{ket}(+)^{\otimes N}$.

#### Solution : scaling & Benchmarking

Our solution requires $N^2$ qubits, where $N$ is the number of nodes of our graph.

### Heading towards the summits : a real challenge

Enthrilled by this new powerful formalism, we tackle more complex trails. We consider now an 5-nodes graph, whose starting and ending points are fixed. However, we are now cruelly missing qubits, we need to be smart...

#### Back to familiar problems

We first realise that new problem of size N=5 nodes could be easily map to our previous problem, while reducing the number of nodes to 3!

Considering the TSP on a N-node graph with fixed starting and ending point, we can transform our cost hamiltonian to simplify our problem to a normal TSP on a (N-2)-node graph. This consist in noting that the elements $x_{i,t}$ are all determined for $i \in {0, N-1}$ and $t \in {0, N-1}$, so that we can reduce the Hilbert space of our solutions to $i \in \{1, 2,..., N-2\}$ and $t \in \{1, 2,..., N-2\}$ (encoded in $(N-2)^2$ qubits) by adding the following terms in the cost hamiltonian:

$$C_{\text{ini}} = \sum_{i=1}^{N-2} w_{1, i} x_{i, 1}, \\
C_{\text{fin}} = \sum_{i=1}^{N-2} w_{i, N-1} x_{i, N-2}.$$

The first term $C_{\text{ini}}$ corresponds to the weight of the first taken edge, and the second term $C_{\text{fin}}$ to the weight of the last taken edge.

So far, our solution requires $(N-2)^2=9$ qubits.

#### Taking advantage of the redundancy in the conditions

Limited to 6 qubits by our current quantum computers, our team is still looking for a slight but decisive improvement of our approach. We then realize that our encoding presented actually a redundancy, which we could use to reduce once more the number of required qubits...

Disclaimer : in the following, n=N-2 is the number of nodes of the subgraph, obtained from our intial graph by removing the starting and ending points. We kept the notation $\{w_{i,j}, \text{ for } i,j\in\{0,...,n-1\}\}$ to describle the weights of this subgraph.

Once considering a graph of n nodes with no specified starting and ending points, we can further reduce the number of required qubits by noting that once all but one nodes are selected, the last node of the tour is fully determined. This property comes from the condition
$$ \forall t, \sum_{i=1}^{n-1}x_{i,t} = 1, $$
which reduces the space of feasible solutions to a Hilbert space of size $n(n-1)$ (the $n$ factor coming from $t\in\{0, 1,..., n-1\}$ and the $n-1$ factor from $i\in\{0, 1,... n-2\}$).

We thus consider the above constraint on steps fully satisfied and set 
$$\forall t, x_{n-1, t} = 1 - \sum_{i=0}^{n-2}x_{i,t}.$$
Doing so, the main term of the cost Hamiltonian reads:

$$C_0(x) = \sum_{i,j=0}^{n-2}w_{i,j}\sum_{t=0}^{n-1}x_{i,t}x_{j,t+1} + 2\sum_{j=0}^{n-2}w_{n-1,j}\sum_{t=0}^{n-1}(1-\sum_{k=0}^{n-2}x_{k,t})x_{j,t+1},$$

and the constraint on nodes:

$$C_1(x) = \sum_{i=0}^{n-2}(1-\sum_{t=0}^{n-1}x_{i,t})^2 + (\sum_{k=0}^{n-2}\sum_{t=0}^{n-1}x_{i,t} - (n-1))^2.$$

Zooming out and remembering that our n-nodes graph is part of a (n+2)-nodes graph with fixed starting and ending points, we also need to transform the costs from the initial and final edges:

$$C_{\text{ini}}(x) = \sum_{i=0}^{n-2}(w_{-1, i} - w_{-1, n-1})x_{i,0} + w_{-1, n-1},$$

$$C_{\text{fin}}(x) = \sum_{i=0}^{n-2}(w_{i, n} - w_{n-1, n})x_{i,n-1} + w_{n-1, n}.$$

#### Success of the mission ? 

In the interpretation of our output state, we

Scaling :

$(N-2)(N-3)$ required qubits, which is equal to 6 for $N=5$ !

Results :

We compared the results obtained with our algorithm with the route calculated in the classical way, and we noticed that our algorithm followed a different path.

Benchmarking:

The quantity used fot the benchmarking is the fidelity, one way to evalute is, given the known ground state encoding the solution $|\psi_{\text{GS}}\rangle$ and the obtained solution $|\psi_{\text{sol}}\rangle$, the fidelity is evaluated as

$$ F = |\langle \psi_{\text{GS}}|\psi_{\text{sol}}\rangle|^2$$

#### Prospects for our next adventure

Both excited by the prospectives that our new quantum solvers offer to our team, as well as frustated that they are bounded, in the middle-term future, to a limited number of qubits, we would like to find a way to arbitrarily reduce the number of required qubits to solve our problem. Finally, we thought of a hybrid approach, to suppport our quantum device with a classical computer, such that a part of the quantum complexity would be transformed in some classical complexity. Indeed, in the NISQ area we can't rely on arbitrarily big quantum computers: the number of avalaible qubits is in reality resticted to some upper value. Considering an arbitrarily big $N$-node graph, we would thus like to find a way to divide our problem into smaller problems that could be solved by our NISQ computer. To do so, we will carry out a divide-and-conquer hybrid approach.

Considering a N-node graph $G$ with starting and ending points $0$ and $N-1$, we scan every node $i$ that is connected to $0$ and look for the best solution of the TSP on the subgraph $G'=G/\{0\}$ starting from node $i$ and ending at $N-1$. Every of these subproblems requires $(N'-2)(N'-3)$ qubits, with $N'=N-1$. Supposing that we have solved this subproblems and obtains the respective optimal costs $C_i$ and optimal paths $S_i = [i,..., N-1]$, we deduce the global optimal cost as being $\max\{C_i + w_{0,i} \text{ for } i \in\{1,...,N-2\}\}$, reached for $i=i_0$. The optimal path is finally $[0]+S_{i_0}$.

## Hardware part

The goal of this part is to understand the hardware protocol that performs the annealing procedure. We implemented an arbitrary instance of the Set Packing Problem with 4 qubits.

### Problem statement and description 

Consider a set $U=\{1,...6\}$ and a set $\mathcal{V} = \{V_1, V_2, V_3, V_4\}$ of its subsets, where $V_1=\{1, 2, 5\}$, $V_2=\{4, 6\}$, $V_3=\{2, 4\}$, $V_4=\{1, 2, 3, 6\}$. We are looking for the biggest subset $\mathcal{R}$ of $\mathcal{V}$ such that all elements of $\mathcal{R}$ are disjoint.

We define the binary variables $\{x_i, \text{ for } i\in\{1,...,4\}\}$ as $x_i = 1$ if $V_i \in \mathcal{R}$ and $0$ otherwise.
The constraint of disjonction is encoded in the following Hamiltonian:
$$H_A = A\sum_{i,j=1}^{N}x_ix_j\delta_{i,j}$$
where $\delta_{i,j}=1$ if $V_i$ and $V_j$ are non disjoint, $0$ otherwise:
$$\Delta = (\delta_{i,j})=\begin{pmatrix} 0 & 0 & 1 & 1 \\ 0 & 0 & 1 & 1 \\ 1 & 1 & 0 & 1 \\ 1 & 1 & 1 & 0 \end{pmatrix},$$
while the maximal size of $\mathcal{R}$ is encoded in:
$$H_B = -B\sum_{i=1}^{N}x_i.$$
We need $B<A$ to ensure that the disjonction constraint is well satisfied. We set e.g. $B=2$ and $A=4$.

We then transform this problem into an Ising form, using the variables $\{z_i\}$ such that $x_i=\frac{1-z_i}{2}$. In these variables, the Hamiltonians $H_A$ and $H_B$ read:
$$H_A = - 2 \sum_{i=1}^{N}(\sum_{j=1}^{N}\delta_{i,j})z_i + \sum_{i,j=1}^{N}z_iz_j\delta_{i,j},$$
$$H_B = \sum_{i=1}^{N}z_i,$$
with a constant that does not impact our solution.

Therefore, our problem comes down to the Ising Hamiltonian $H = H_A + H_B$ with Pauli coefficients:
$$h_i^z = 1 - 2 \sum_{j=1}^{N}\delta_{i,j},$$
$$J_{i,j} = \delta_{i,j}.$$

### Circuit encoding

The picture named "layout" in our repository shows a schematic description of the layout of the corresponding circuit. We make sure to space the coupling elements from one another.

### Annealing procedure

We went for a linear annealing procedure, starting from a fully transversed field towards our target Hamiltonian.